In [1]:
import zipfile
with zipfile.ZipFile('/storage/BW2/data/simpsons_dataset.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/storage/BW2/data')

In [2]:
import pandas as pd

df = pd.read_csv('/storage/BW2/data/simpsons_dataset.csv')

In [2]:
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [3]:
df.isnull().sum()

raw_character_text    17814
spoken_words          26459
dtype: int64

In [4]:
df.dtypes

raw_character_text    object
spoken_words          object
dtype: object

In [7]:
df['raw_character_text'].value_counts().head(15)

Homer Simpson             29782
Marge Simpson             14141
Bart Simpson              13759
Lisa Simpson              11489
C. Montgomery Burns        3162
Moe Szyslak                2862
Seymour Skinner            2438
Ned Flanders               2144
Grampa Simpson             1880
Milhouse Van Houten        1862
Chief Wiggum               1830
Krusty the Clown           1768
Nelson Muntz               1172
Lenny Leonard              1166
Apu Nahasapeemapetilon     1006
Name: raw_character_text, dtype: int64

In [8]:
df.shape

(158314, 2)

In [3]:
df = df.dropna().reset_index(drop=True)

In [4]:
df.shape

(131853, 2)

In [ ]:
df['raw_character_text'].value_counts().tail(30)

In [5]:
# df['raw_character_text'] = df['raw_character_text'].value_counts()[df['raw_character_text'].value_counts()>1]

In [6]:
df.shape

(131853, 2)

In [9]:
df = df[df.groupby('raw_character_text')['raw_character_text'].transform('size') > 1]

In [11]:
df['raw_character_text'].value_counts()

Homer Simpson          29782
Marge Simpson          14141
Bart Simpson           13759
Lisa Simpson           11489
C. Montgomery Burns     3162
                       ...  
Homer Glumpet              2
Small Kid                  2
Huge Man #1                2
Performer                  2
Postal Clerk #2            2
Name: raw_character_text, Length: 3713, dtype: int64

In [10]:
df.shape

(129059, 2)

In [13]:
df.nunique()

raw_character_text      3713
spoken_words          119271
dtype: int64

### tokenize

In [ ]:
!pip install gensim

In [ ]:
!pip install spacy

In [5]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy

In [ ]:
# # Tokenizer function

# def tokenize(text):
#     tokens = []
#     for token in simple_preprocess(text):
#         if token not in STOPWORDS and len(token) > 3:
#             tokens.append(token)
#     return tokens

In [6]:
nlp = spacy.load('en_core_web_lg')
STOPWORDS = nlp.Defaults.stop_words.union({' ', ''})

In [7]:
def tokenize(doc):
    text = re.sub(r'[^a-zA-Z ]', '', doc)
    text = text.lower()
    tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab)
    tokens = tokenizer(text)
    list_of_tokens = [t for t in tokens if (str(t) not in STOPWORDS) and (t.is_punct == False)]
    return (list_of_tokens)

In [8]:
df['tokens'] = df['spoken_words'].apply(tokenize)

In [9]:
df['tokens'].head()

0    [actually, little, disease, magazines, news, s...
1                              [wheres, mr, bergstrom]
2    [dont, know, id, sure, like, talk, didnt, touc...
3                                [life, worth, living]
4    [polls, open, end, recess, case, decided, thou...
Name: tokens, dtype: object

In [ ]:
df.shape

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words=STOPWORDS, lowercase=True, min_df=0.02, max_df=0.95)

In [79]:
# Create a vocabulary and get word counts per document
sparse = tfidf.fit_transform(df['spoken_words'])

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,bart,don,good,got,hey,homer,know,let,like,ll,man,oh,right,think,uh,ve,want,yeah
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.553429,0.0,0.0,0.0,0.0,0.602737,0.0,0.574826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
# Instantiate kNN
from sklearn.neighbors import NearestNeighbors

# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
# # Query Using kneighbors 
# nn.kneighbors([dtm.iloc[0]])

In [92]:
in_text = 'stuck in a loop with no where to go please explain'

# in_text = 'I have mispaced my'
# Query for similiar quotes...
newz = tfidf.transform([in_text])

finders = nn.kneighbors(newz.todense())

In [93]:
q = finders[1][0]
q

array([ 155, 2635, 1329,   67,  319])

In [94]:
for i in p:
    print(df['spoken_words'][i])
    

Why not? I paid for it.
Hmm.
In the Cincinnati - Miami game, I declare Cincinnati to be my "Shoe-in of the Week".
Nothing.
I see. I also understand bowling expressions.


## Generate Fake Quotes

In [9]:
# First to filter by character... 
homer_opus = df[df['raw_character_text'] == 'Homer Simpson'].reset_index()

In [ ]:
homer_opus = homer_opus.drop(columns='index')

In [ ]:
homer_opus.head()

In [ ]:
# np.savetxt(r'/storage/BW2/data/np.txt', homer_opus['spoken_words'], fmt='str')

In [ ]:
pd.set_option('display.max_colwidth', 600)

In [ ]:
homer_opus.shape

In [ ]:
homer_opus['spoken_words'] = homer_opus['spoken_words'].astype('str')
mask = (homer_opus['spoken_words'].str.len() > 64)
homer_opus = homer_opus.loc[mask]
homer_opus.head()

In [ ]:
homer_opus.shape

In [ ]:
homer_opus.dtypes

In [ ]:
# with open('/storage/BW2/c.txt', 'a') as f:
#     f.write(str(homer_opus['spoken_words']))

In [ ]:
# put all the quotes in a single text file... DID IT ALREADY...

homer_opus['spoken_words'].to_csv('/storage/BW2/c.txt', header=None, index=None, sep=' ', mode='a')

In [5]:
homer_opus.shape

NameError: name 'homer_opus' is not defined

In [ ]:
!pip install keras

In [10]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [14]:
# load ascii text and covert to lowercase
filename = 'b.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [15]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [16]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  1509125
Total Vocab:  69


In [17]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 64
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  1509061


In [18]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# # define the checkpoint
# filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

In [ ]:
# Smaller model...

# model.fit(X, y, epochs=1, batch_size=128, callbacks=callbacks_list)

In [ ]:
# # load the network weights
# filename = "weights-improvement-01-1.8425.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# import sys

In [ ]:
# # pick a random seed
# start = numpy.random.randint(0, len(dataX)-1)
# pattern = dataX[start]
# print ("Seed:")
# print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# # generate characters
# for i in range(1000):
# 	x = numpy.reshape(pattern, (1, len(pattern), 1))
# 	x = x / float(n_vocab)
# 	prediction = model.predict(x, verbose=0)
# 	index = numpy.argmax(prediction)
# 	result = int_to_char[index]
# 	seq_in = [int_to_char[value] for value in pattern]
# 	sys.stdout.write(result)
# 	pattern.append(index)
# 	pattern = pattern[1:len(pattern)]
# print("\nDone.")

In [17]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# More LAyerS!!! # Define bigger model

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.10))
model.add(LSTM(256))
model.add(Dropout(0.10))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-opus_c.hdf5"

In [ ]:
# fit the model
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Train on 844466 samples
Epoch 1/50
844416/844466 [============================>.] - ETA: 0s - loss: 2.5932
Epoch 00001: loss did not improve from 1.54467
844466/844466 [==============================] - 356s 422us/sample - loss: 2.5932
Epoch 2/50
844416/844466 [============================>.] - ETA: 0s - loss: 2.2450
Epoch 00002: loss did not improve from 1.54467
844466/844466 [==============================] - 354s 419us/sample - loss: 2.2450
Epoch 3/50
844416/844466 [============================>.] - ETA: 0s - loss: 2.0774
Epoch 00003: loss did not improve from 1.54467
844466/844466 [==============================] - 354s 419us/sample - loss: 2.0774
Epoch 4/50
844416/844466 [============================>.] - ETA: 0s - loss: 1.9779
Epoch 00004: loss did not improve from 1.54467
844466/844466 [==============================] - 353s 419us/sample - loss: 1.9779
Epoch 5/50
844416/844466 [============================>.] - ETA: 0s - loss: 1.9081
Epoch 00005: loss did not improve from 1.5446

In [21]:
# load the network weights
filename = "weights-improvement-50-1.5447.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [22]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [23]:
import sys

In [4]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(128):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

NameError: name 'numpy' is not defined

In [ ]:
# SAVE for comparison

In [ ]:
# # fit the model
# model.fit(X, y, epochs=40, batch_size=64, callbacks=callbacks_list)